# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
%%time
data = get_date_list(data)
data = read_data(data)

Read dataframe with 16797607 records.
Dataframe has 115071 records between (datetime.datetime(2021, 1, 1, 0, 0), datetime.datetime(2021, 1, 4, 0, 0)).
Record num : 115071
Wall time: 15.9 s


In [3]:
for data['i'] in tqdm(range(0, data['df_len'])):
    
    data['df']['DateTime_frmt'][data['i']] = dt.datetime.strptime(data['df']['DateTime'][data['i']].split(".")[0],"%Y%m%d %H:%M:%S")
    
    data['ask'] = data['df']['Ask'][data['i']]
    data['bid'] = data['df']['Bid'][data['i']]  
    
    data['tick'] = (data['df']['Ask'][data['i']] + data['df']['Bid'][data['i']])/2
    data['df']['tick'][data['i']] = data['tick']
        
    # sema before after loops --------------------------------
    if len(data['sema_list']) < data['sema_len']:
        data =  before_sema(data)
        continue

    if len(data['sema_list']) == data['sema_len']:
        data = after_sema(data)                 
    # ----------------------------------------------------------

#     # slema before after loops --------------------------------
#     if len(data['slema_list']) < data['slema_len']:
#         data =  before_slema(data)
#         continue

#     if len(data['slema_list']) == data['slema_len']:
#         data = after_slema(data)                 
#     # ----------------------------------------------------------    
    
    
#     # lema before after loops --------------------------------
#     if len(data['lema_list']) < data['lema_len']:
#         data =  before_lema(data)
#         continue

#     if len(data['lema_list']) == data['lema_len']:
#         data = after_lema(data)                 
#     # ----------------------------------------------------------
    
    data = get_ohlc(data)    
    data = get_trend_fwd(data)    
    data = get_position(data)
    
    if not data['position']:
        continue
        
    # Get Dirs --------------------------------
    if len(data['dir_list']) < 2:
        data['dir_list'].append(data['position'])   
        continue

    elif len(data['dir_list']) == 2:
        data = get_cross_dir(data)
    # ----------------------------------------------------------  
    
    data = calculate_pl(data)
    data = simple_take_profit(data)
    data = tick_close(data)
    data = simple_stop_loss(data)
    data = make_order(data)

 75%|█████████████████████████▌        | 86540/115071 [08:23<03:08, 151.05it/s]

Sum : -0.00457
-----------------------------
                    DateTime       pl
29152  20210104 06:58:24.817 -0.00088
54673  20210104 12:03:04.234 -0.00090
62138  20210104 13:24:58.169 -0.00088
73278  20210104 14:53:38.526 -0.00091
86525  20210104 16:14:10.019 -0.00100


100%|█████████████████████████████████| 115071/115071 [11:28<00:00, 167.09it/s]


In [4]:
num_rec = len(data["df"])
hours = np.round((list(data['df']['DateTime_frmt'])[-1] - list(data['df']['DateTime_frmt'])[0]).total_seconds() / 60 / 60 , 2)
recsphour = int(num_rec/hours)

print(f'Records   : {num_rec}')
print(f'Hours     : {hours}')
print(f'Recs/hour : {recsphour}')
print(f'Recs/min  : {int(recsphour/60)}')
print(f'Recs/sec  : {np.round(recsphour/3600, 1)}')

data['df'].head()

Records   : 115071
Hours     : 26.0
Recs/hour : 4425
Recs/min  : 73
Recs/sec  : 1.2


,DateTime,Bid,Ask,tick,DateTime_frmt,sema,slema,lema,position,to_order,...,stop_loss_pip,pl_move_trail_trigger,pl_move_min,h,l,long_open,long_close,short_open,short_close,pl
0,20210103 22:00:00.040,1.22396,1.22411,1.224035,2021-01-03 22:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20210103 22:00:00.144,1.22395,1.22410,1.224025,2021-01-03 22:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20210103 22:00:00.434,1.22393,1.22411,1.224020,2021-01-03 22:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20210103 22:00:00.738,1.22393,1.22410,1.224015,2021-01-03 22:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20210103 22:00:00.933,1.22392,1.22409,1.224005,2021-01-03 22:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
data['plot_trend_calc_lines']   = False

In [56]:
plot_graph(data)

In [6]:
data['df'].to_csv('data/temp.csv')

In [7]:
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)
send_telegram_message(f'------------\n{sum(pl_list)}\n------------\n{pl_list}\n------------')

NameError: name 'pl_list' is not defined

In [47]:
data['df']['std'] = np.nan
data['df']['std_up'] = np.nan
data['df']['std_down'] = np.nan

In [48]:
for i in tqdm(np.arange(1800, len(data['df']))):
    data['df']['std'].loc[i] = np.std(data['df']['tick'][i-1800:i])

100%|████████████████████████████████| 113271/113271 [00:45<00:00, 2478.18it/s]


In [49]:
data['df']['std_up'] = data['df']['tick'] + data['df']['std']
data['df']['std_down'] = data['df']['tick'] - data['df']['std']

In [50]:
data['df'][['tick', 'std', 'std_up', 'std_down']]

,tick,std,std_up,std_down
0,1.224035,NaN,NaN,NaN
1,1.224025,NaN,NaN,NaN
2,1.224020,NaN,NaN,NaN
3,1.224015,NaN,NaN,NaN
4,1.224005,NaN,NaN,NaN
...,...,...,...,...
115066,1.225255,0.000136,1.225391,1.225119
115067,1.225265,0.000136,1.225401,1.225129
115068,1.225275,0.000136,1.225411,1.225139
115069,1.225270,0.000136,1.225406,1.225134


In [42]:
np.max(data['df']['std'][data['df']['std'].notnull()])

0.000512593553761673